In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [4]:
%%sql
SELECT * FROM facts LIMIT 5

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- name — the name of the country.
- area— the country's total area (both land and water).
- area_land — the country's land area in square kilometers.
- area_water — the country's waterarea in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people.

Which is the min pouplation?

In [15]:
%%sql
SELECT MIN(population) population FROM facts WHERE population > 0

Done.


population
48


Which is the max population?

In [11]:
%%sql
SELECT MAX(population) population FROM facts

Done.


population
7256490011


Which is the MIN population growth?

In [17]:
%%sql
SELECT MIN(population_growth) population FROM facts WHERE population_growth > 0

Done.


population
0.01


Which is the MAX population growth?

In [13]:
%%sql
SELECT MAX(population_growth) population FROM facts

Done.


population
4.02


Which is the Country with the Min population?

In [20]:
%%sql
SELECT name FROM facts WHERE population IN (
    SELECT MIN(population) population FROM facts
)


Done.


name
Antarctica


Which is the Country with the Max population?

In [21]:
%%sql
SELECT name FROM facts WHERE population IN (
    SELECT MAX(population) population FROM facts
)

Done.


name
World


It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for [Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html):

[<img src="https://s3.amazonaws.com/dq-content/257/fb_antarctica.png">](https://s3.amazonaws.com/dq-content/257/fb_antarctica.png/)


Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

Which is the country with the max population?

In [24]:
%%sql
SELECT name FROM facts WHERE population IN (
    SELECT MAX(population) population FROM facts WHERE name NOT IN ('World')
)

Done.


name
China


Which is the country with the min population?

In [26]:
%%sql
SELECT name FROM facts WHERE population IN (
    SELECT MIN(population) population FROM facts WHERE name NOT IN ('Antarctica')
)

Done.


name
Pitcairn Islands


Which is the countries with the min population growth?

In [29]:
%%sql
SELECT name 
FROM facts 
WHERE population_growth IN (
    SELECT MIN(population_growth) population 
    FROM facts WHERE name NOT IN ('Antarctica')
)

Done.


name
Holy See (Vatican City)
Cocos (Keeling) Islands
Greenland
Pitcairn Islands


Which is the countries with the max population growth?

In [30]:
%%sql
SELECT name 
FROM facts 
WHERE population_growth IN (
    SELECT MAX(population_growth) population 
    FROM facts WHERE name NOT IN ('World')
)

Done.


name
South Sudan


What is the population and area average?

In [35]:
%%sql
SELECT AVG(population) avg_population,  AVG(area) avg_area
FROM facts

Done.


avg_population,avg_area
62094928.32231405,555093.546184739


Which countries are below the population average?

In [37]:
%%sql
SELECT name FROM facts WHERE population < (
    SELECT AVG(population) avg_population FROM facts
)


Done.


name
Afghanistan
Albania
Algeria
Andorra
Angola
Antigua and Barbuda
Argentina
Armenia
Australia
Austria


Which countries are below the area average?

In [38]:
%%sql
SELECT name FROM facts WHERE area < (
    SELECT AVG(area) avg_population FROM facts
)

Done.


name
Albania
Andorra
Antigua and Barbuda
Armenia
Austria
Azerbaijan
"Bahamas, The"
Bahrain
Bangladesh
Barbados


Which countries have a higher death rate than birth rate?

In [40]:
%%sql
SELECT name 
FROM facts 
WHERE death_rate > birth_rate

Done.


name
Austria
Belarus
Bosnia and Herzegovina
Bulgaria
Croatia
Czech Republic
Estonia
Germany
Greece
Hungary
